# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [121]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [122]:
# Reaing the csv file
cities_csv = "../Weatherpy/output_data/weather_city_data.csv"
weather_cities = pd.read_csv(cities_csv)
weather_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tautira,-17.7333,-149.1500,89.60,58,40,4.61,pf,1619747284
1,humaita,-7.5061,-63.0208,73.54,97,100,2.64,br,1619747302
2,bethel,41.3712,-73.4140,68.00,77,75,5.75,us,1619747071
3,rikitea,-23.1203,-134.9692,76.78,66,87,12.68,pf,1619747022
4,cayenne,4.9333,-52.3333,77.00,94,0,4.29,gf,1619747303
...,...,...,...,...,...,...,...,...,...
548,murgab,37.4966,61.9714,66.20,77,90,9.22,tm,1619747430
549,franca,-20.5386,-47.4008,58.33,61,99,5.32,br,1619747517
550,orlik,52.5178,99.8250,35.91,60,100,4.14,ru,1619747517
551,valday,57.9747,33.2525,34.84,94,100,3.62,ru,1619747517


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [123]:
# Configuring the gmaps
gmaps.configure(api_key=g_key)

In [124]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_cities[["Lat","Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_cities["Humidity"].astype(float)

In [125]:
locations

,Lat,Lng
0,-17.7333,-149.1500
1,-7.5061,-63.0208
2,41.3712,-73.4140
3,-23.1203,-134.9692
4,4.9333,-52.3333
...,...,...
548,37.4966,61.9714
549,-20.5386,-47.4008
550,52.5178,99.8250
551,57.9747,33.2525


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [126]:
# DataFrame to find ideal weather condition
# A max temperature lower than 80 degrees but higher than 70.

ideal_weather = weather_cities.loc[
    (weather_cities["Max Temp"] >=70) & 
    (weather_cities["Max Temp"] <=80)&
    (weather_cities["Wind Speed"] <=10)&
    (weather_cities["Cloudiness"] == 0)]

len(ideal_weather)


# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
ideal_weather= ideal_weather.dropna()

In [127]:
ideal_weather.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4,cayenne,4.9333,-52.3333,77.00,94,0,4.29,gf,1619747303
1,71,zharkent,44.1628,80.0000,75.49,18,0,5.44,kz,1619747331
2,91,manicore,-5.8092,-61.3003,73.40,100,0,1.86,br,1619747339
3,129,weihe,32.9952,105.3247,72.41,42,0,1.97,cn,1619747356
4,219,beloha,-25.1667,45.0500,70.75,68,0,5.53,mg,1619747392
5,260,alenquer,-1.9417,-54.7383,75.20,100,0,3.09,br,1619747408
6,280,santa isabel,28.3833,-113.3500,72.25,20,0,6.73,mx,1619747416
7,319,riyadh,24.6877,46.7219,71.60,28,0,3.44,sa,1619747279
8,339,haikou,20.0458,110.3417,77.00,83,0,2.24,cn,1619747439
9,350,jiujiang,29.7333,115.9833,78.87,54,0,7.74,cn,1619747444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [135]:
# Store into variable named hotel_df.
hotel_df = ideal_weather.reset_index()

In [136]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [137]:
# Hit the Google Places API for each city's coordinates.

# set up a parameters dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

curr_hotel
# Looping through all the information retrieving
for index,row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [138]:
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,4,cayenne,4.9333,-52.3333,77.00,94,0,4.29,gf,1619747303,Hôtel Le Dronmi
1,71,zharkent,44.1628,80.0000,75.49,18,0,5.44,kz,1619747331,гостиница Satti
2,91,manicore,-5.8092,-61.3003,73.40,100,0,1.86,br,1619747339,Hotel Durval
3,129,weihe,32.9952,105.3247,72.41,42,0,1.97,cn,1619747356,
4,219,beloha,-25.1667,45.0500,70.75,68,0,5.53,mg,1619747392,
5,260,alenquer,-1.9417,-54.7383,75.20,100,0,3.09,br,1619747408,Josilene Castro
6,280,santa isabel,28.3833,-113.3500,72.25,20,0,6.73,mx,1619747416,
7,319,riyadh,24.6877,46.7219,71.60,28,0,3.44,sa,1619747279,"InterContinental Riyadh, an IHG Hotel"
8,339,haikou,20.0458,110.3417,77.00,83,0,2.24,cn,1619747439,Sun City Restaurant
9,350,jiujiang,29.7333,115.9833,78.87,54,0,7.74,cn,1619747444,Tianhai Fashion Theme Hotel


In [139]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [142]:
# Add marker layer ontop of heat map
# Set parameters to search for hotels with 5000 meters.
locations.head()
# dataframe with columns ('latitude', 'longitude', 'magnitude')
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heatmap_layer)
fig.add_layer(markers)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [102]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5
